# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('output_data/cities.csv')
city_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Grandview,38.89,-94.53,34.00,86,90,0.09,US,1603849845
1,Punta Arenas,-53.15,-70.92,42.80,75,0,26.40,CL,1603849756
2,Ushuaia,-54.80,-68.30,46.40,36,75,12.75,AR,1603849761
3,Gushikawa,26.35,127.87,82.40,54,20,13.87,JP,1603849847
4,Saint-Pierre,-21.34,55.48,69.80,64,0,4.70,RE,1603849794
...,...,...,...,...,...,...,...,...,...
571,Dordrecht,51.81,4.67,51.01,89,63,5.01,NL,1603850125
572,Pedernales,18.04,-71.74,80.67,71,3,0.56,DO,1603850125
573,San Roque,36.21,-5.38,60.80,67,0,13.87,ES,1603850126
574,Huarmey,-10.07,-78.15,63.79,86,92,6.44,PE,1603850126


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
locations = city_df[['Lat','Lng']].astype(float)
weight = city_df['Humidity']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=weight, 
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = city_df.loc[(city_df['Max Temp'] > 80) & (city_df['Max Temp'] < 90) & (city_df['Humidity'] < 70) 
                       & (city_df['Humidity'] > 50) & (city_df['Cloudiness'] < 50),:]
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,Gushikawa,26.35,127.87,82.40,54,20,13.87,JP,1603849847
37,Roebourne,-20.78,117.13,81.63,54,33,11.16,AU,1603849866
40,Butaritari,3.07,172.79,81.09,66,22,4.29,KI,1603849868
94,Kupang,-10.17,123.58,87.96,61,20,7.52,ID,1603849893
162,Padang,-0.95,100.35,84.34,60,23,3.04,ID,1603849886
255,Kendari,-3.94,122.50,88.92,53,15,4.68,ID,1603849955
276,Nishihara,26.18,127.76,82.40,54,20,13.87,JP,1603849782
304,Kimbe,-5.55,150.14,87.67,63,42,5.77,PG,1603849995
310,Nhulunbuy,-12.23,136.77,89.60,55,40,12.75,AU,1603849998
312,Port Hedland,-20.32,118.57,82.40,58,32,8.05,AU,1603849999


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
hotel_df = ideal_df.copy()
hotel_df['Hotel Name'] = ''
params = {
    'key': g_key,
    'types': 'lodging',
    'radius': 5000
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    coord = f'{lat}, {lng}'
    
    params['location'] = coord
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except IndexError:
        hotel_df.loc[index, 'Hotel Name'] = 'None'
    


In [16]:
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != 'None',:]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,Gushikawa,26.35,127.87,82.40,54,20,13.87,JP,1603849847,Kirari Guest House
37,Roebourne,-20.78,117.13,81.63,54,33,11.16,AU,1603849866,Harding River Caravan Park
40,Butaritari,3.07,172.79,81.09,66,22,4.29,KI,1603849868,Isles Sunset Lodge
94,Kupang,-10.17,123.58,87.96,61,20,7.52,ID,1603849893,Sylvia Hotel Premier Kupang
162,Padang,-0.95,100.35,84.34,60,23,3.04,ID,1603849886,Mercure Padang
255,Kendari,-3.94,122.50,88.92,53,15,4.68,ID,1603849955,Hotel Wixel
276,Nishihara,26.18,127.76,82.40,54,20,13.87,JP,1603849782,Minshuku Agaihama
304,Kimbe,-5.55,150.14,87.67,63,42,5.77,PG,1603849995,Kimbe Bay Hotel
310,Nhulunbuy,-12.23,136.77,89.60,55,40,12.75,AU,1603849998,East Arnhem Real Estate
312,Port Hedland,-20.32,118.57,82.40,58,32,8.05,AU,1603849999,The Esplanade Hotel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))